In [ ]:
import pandas as pd
import json
from pathlib import Path

# Load the CSV file from the data directory
data_dir = Path("../data")
file_path = data_dir / "synthetic-it-call-center-tickets.csv"

df = pd.read_csv(file_path).fillna("")

df_1000 = df.head(1000)

def normalize_keys(row):
    return {
        "category": row["category"],
        "subcategory": row["subcategory"],
        "issue_request": row["issue/request"],
        "assignment_group": row["assignment_group"],
        "software_system": row["software/system"],
        "poor_close_notes": row["poor_close_notes"],
        "resolution_time": row["resolution_time"],
        "info_score_close_notes": row["info_score_close_notes"]
    }

with open("finetune_multioutput_small.jsonl", "w", encoding="utf-8") as f:
    for _, row in df_1000.iterrows():

        # Entrada do modelo
        user_msg = (
            f"Short description: {row['short_description']}\n"
            f"Content: {row['content']}\n"
            "Preencha os seguintes campos em JSON:"
            "\n- category\n- subcategory\n- issue_request\n- assignment_group"
            "\n- software_system\n- poor_close_notes\n- resolution_time\n- info_score_close_notes"
        )

        # Saída esperada do modelo (labels)
        assistant_msg = json.dumps(normalize_keys(row), ensure_ascii=False)

        record = {
            "messages": [
                {"role": "user", "content": user_msg},
                {"role": "assistant", "content": assistant_msg}
            ]
        }

        f.write(json.dumps(record, ensure_ascii=False) + "\n")
